In [1]:
import cirq

In [2]:
help(cirq.CZTargetGateset)

Help on class CZTargetGateset in module cirq.transformers.target_gatesets.cz_gateset:

class CZTargetGateset(cirq.transformers.target_gatesets.compilation_target_gateset.TwoQubitCompilationTargetGateset)
 |  CZTargetGateset(*, atol: float = 1e-08, allow_partial_czs: bool = False, additional_gates: Sequence[Union[Type[ForwardRef('cirq.Gate')], ForwardRef('cirq.Gate'), ForwardRef('cirq.GateFamily')]] = (), preserve_moment_structure: bool = True) -> None
 |  
 |  Target gateset accepting CZ + single qubit rotations + measurement gates.
 |  
 |  By default, `cirq.CZTargetGateset` will accept and compile unknown gates to
 |  the following universal target gateset:
 |  - `cirq.CZ` / `cirq.CZPowGate`: The two qubit entangling gate.
 |  - `cirq.PhasedXZGate`: Single qubit rotations.
 |  - `cirq.MeasurementGate`: Measurements.
 |  - `cirq.GlobalPhaseGate`: Global phase.
 |  
 |  Optionally, users can also specify additional gates / gate families which should
 |  be accepted by this gateset via 

In [4]:
import cirq

# Step 1: Create a quantum circuit
qubits = [cirq.GridQubit(0, 0), cirq.GridQubit(0, 1)]
circuit = cirq.Circuit(
    cirq.H(qubits[0]),
    cirq.CNOT(qubits[0], qubits[1]),
    cirq.Y(qubits[0])**0.5,
    cirq.measure(*qubits, key='m')
)

# Step 2: Decompose the circuit to the desired basis set
def decompose_to_basis(circuit, basis_gates):
    decomposed_circuit = cirq.Circuit()
    for op in circuit.all_operations():
        try:
            decomposed_op = cirq.decompose(op, keep=lambda g: g in basis_gates)
            decomposed_circuit.append(decomposed_op)
        except ValueError:
            # If decomposition fails, you can either approximate or skip
            decomposed_circuit.append(op)
    return decomposed_circuit

# Define your desired basis gates
basis_gates = [cirq.rx, cirq.ry, cirq.rz, cirq.CX]

# Transpile the circuit
transpiled_circuit = decompose_to_basis(circuit, basis_gates)

# Step 3: Print the transpiled circuit
print("Original Circuit:")
print(circuit)

print("\nTranspiled Circuit:")
print(transpiled_circuit)


Original Circuit:
(0, 0): ───H───@───Y^0.5───M('m')───
               │           │
(0, 1): ───────X───────────M────────

Transpiled Circuit:
(0, 0): ───H───@───Y^0.5───M('m')───
               │           │
(0, 1): ───────X───────────M────────


In [5]:
import cirq
import numpy as np

# Step 1: Create a quantum circuit
qubits = [cirq.GridQubit(0, 0), cirq.GridQubit(0, 1)]
circuit = cirq.Circuit(
    cirq.H(qubits[0]),
    cirq.CNOT(qubits[0], qubits[1]),
    cirq.Y(qubits[0])**0.5,
    cirq.measure(*qubits, key='m')
)

# Function to manually decompose Hadamard and other gates to the desired basis
def custom_decompose(op):
    if isinstance(op.gate, cirq.HPowGate):
        qubit = op.qubits[0]
        return [
            cirq.rx(np.pi).on(qubit),
            cirq.ry(np.pi / 2).on(qubit)
        ]
    return [op]

# Step 2: Decompose the circuit to the desired basis set
def decompose_to_basis(circuit, basis_gates):
    decomposed_circuit = cirq.Circuit()
    for op in circuit.all_operations():
        try:
            decomposed_op = cirq.decompose(op, keep=lambda g: g in basis_gates)
            decomposed_circuit.append(decomposed_op)
        except ValueError:
            # Attempt to manually decompose known gates like Hadamard
            manually_decomposed = custom_decompose(op)
            decomposed_circuit.append(manually_decomposed)
    return decomposed_circuit

# Define your desired basis gates
basis_gates = [cirq.rx, cirq.ry, cirq.rz, cirq.CX]

# Transpile the circuit
transpiled_circuit = decompose_to_basis(circuit, basis_gates)

# Step 3: Print the transpiled circuit
print("Original Circuit:")
print(circuit)

print("\nTranspiled Circuit:")
print(transpiled_circuit)


Original Circuit:
(0, 0): ───H───@───Y^0.5───M('m')───
               │           │
(0, 1): ───────X───────────M────────

Transpiled Circuit:
(0, 0): ───Rx(π)───Ry(0.5π)───@───Y^0.5───M('m')───
                              │           │
(0, 1): ──────────────────────X───────────M────────


In [12]:
help(cirq.transformer)

Help on function transformer in module cirq.transformers.transformer_api:

transformer(cls_or_func: Any = None, *, add_deep_support: bool = False) -> Any
    Decorator to verify API and append logging functionality to transformer functions & classes.
    
    A transformer is a callable that takes as inputs a `cirq.AbstractCircuit` and
    `cirq.TransformerContext`, and returns another `cirq.AbstractCircuit` without
    modifying the input circuit. A transformer could be a function, for example:
    
    >>> @cirq.transformer
    ... def convert_to_cz(
    ...    circuit: cirq.AbstractCircuit, *, context: 'Optional[cirq.TransformerContext]' = None
    ... ) -> cirq.Circuit:
    ...    ...
    
    Or it could be a class that implements `__call__` with the same API, for example:
    
    >>> @cirq.transformer
    ... class ConvertToSqrtISwaps:
    ...    def __init__(self):
    ...        ...
    ...    def __call__(
    ...        self,
    ...        circuit: cirq.AbstractCircuit,
   

In [11]:
print(qc)

[(cirq.Y**0.5).on(cirq.GridQubit(0, 0)), cirq.XPowGate(global_shift=-0.25).on(cirq.GridQubit(0, 0)), (cirq.Y**-0.5).on(cirq.GridQubit(0, 1)), cirq.CZ(cirq.GridQubit(0, 0), cirq.GridQubit(0, 1)), (cirq.Y**0.5).on(cirq.GridQubit(0, 1)), (cirq.Y**0.5).on(cirq.GridQubit(0, 0)), cirq.measure(cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), key=cirq.MeasurementKey(name='m'))]


In [17]:
help(cirq.create_transformer_with_kwargs)


Help on function create_transformer_with_kwargs in module cirq.transformers.target_gatesets.compilation_target_gateset:

create_transformer_with_kwargs(transformer: 'cirq.TRANSFORMER', **kwargs) -> 'cirq.TRANSFORMER'
    Method to capture additional keyword arguments to transformers while preserving mypy type.
    
    Returns a `cirq.TRANSFORMER` which, when called with a circuit and transformer context, is
    equivalent to calling `transformer(circuit, context=context, **kwargs)`. It is often useful to
    capture keyword arguments of a transformer before passing them as an argument to an API that
    expects `cirq.TRANSFORMER`. For example:
    
    >>> def run_transformers(transformers: 'List[cirq.TRANSFORMER]'):
    ...     circuit = cirq.Circuit(cirq.X(cirq.q(0)))
    ...     context = cirq.TransformerContext()
    ...     for transformer in transformers:
    ...         transformer(circuit, context=context)
    ...
    >>> transformers: 'List[cirq.TRANSFORMER]' = []
    >>> tra

In [21]:
from cirq import transformers


In [23]:
help(cirq.optimize_for_target_gateset)

Help on function optimize_for_target_gateset in module cirq.transformers.optimize_for_target_gateset:

optimize_for_target_gateset(circuit: 'cirq.AbstractCircuit', *, context: Optional[ForwardRef('cirq.TransformerContext')] = None, gateset: Optional[ForwardRef('cirq.CompilationTargetGateset')] = None, ignore_failures: bool = True, max_num_passes: Optional[int] = 1) -> 'cirq.Circuit'
    Transforms the given circuit into an equivalent circuit using gates accepted by `gateset`.
    
    Repeat max_num_passes times or when `max_num_passes=None` until no further changes can be done
    1. Run all `gateset.preprocess_transformers`
    2. Convert operations using built-in cirq decompose + `gateset.decompose_to_target_gateset`.
    3. Run all `gateset.postprocess_transformers`
    
    Note:
        The optimizer is a heuristic and may not produce optimal results even with
        max_num_passes=None. The preprocessors and postprocessors of the gate set
        as well as their order yield di

In [50]:
gateset = cirq.Gateset(cirq.ops.common_gates.Rx, cirq.ops.common_gates.Ry, cirq.ops.common_gates.Rz, cirq.CNOT,unroll_circuit_op=True)
      # Optional: Unroll composite gates like cirq.CircuitOperation

qc = cirq.transformer

qc = cirq.optimize_for_target_gateset(circuit, gateset=gateset)

AttributeError: 'Gateset' object has no attribute 'preprocess_transformers'

In [44]:
cirq.CX

cirq.CNOT

In [49]:
cirq.Gateset.name

In [53]:
x=cirq.CompilationTargetGateset([cirq.ops.common_gates.Rx, cirq.ops.common_gates.Ry, cirq.ops.common_gates.Rz, cirq.CNOT],
                              name="Hardware Agnostic",
                              unroll_circuit_op=True,
                              preserve_moment_structure=True)

TypeError: Can't instantiate abstract class CompilationTargetGateset with abstract methods decompose_to_target_gateset, num_qubits

In [65]:
# Apply the default optimization pass
optimized_circuit = cirq.optimize_for_target_gateset(
    circuit,
    gateset=[cirq.ops.common_gates.Rx, cirq.ops.common_gates.Ry, cirq.ops.common_gates.Rz, cirq.CNOT]
)

AttributeError: 'list' object has no attribute 'preprocess_transformers'

In [74]:
qc=circuit
gateset = cirq.CZTargetGateset(allow_partial_czs=True)
optimized_circuit = cirq.optimize_for_target_gateset(qc, gateset=gateset)

In [75]:
print(qc)

(0, 0): ───H───@───Y^0.5───M('m')───
               │           │
(0, 1): ───────X───────────M────────


In [85]:
print(gateset)

Gateset: CZPowTargetGateset
Type GateFamily: cirq.ops.common_gates.CZPowGate
Accepts `cirq.Gate` instances `g` s.t. `isinstance(g, cirq.ops.common_gates.CZPowGate)`

Type GateFamily: cirq.ops.measurement_gate.MeasurementGate
Accepts `cirq.Gate` instances `g` s.t. `isinstance(g, cirq.ops.measurement_gate.MeasurementGate)`

Type GateFamily: cirq.ops.phased_x_z_gate.PhasedXZGate
Accepts `cirq.Gate` instances `g` s.t. `isinstance(g, cirq.ops.phased_x_z_gate.PhasedXZGate)`

Type GateFamily: cirq.ops.global_phase_op.GlobalPhaseGate
Accepts `cirq.Gate` instances `g` s.t. `isinstance(g, cirq.ops.global_phase_op.GlobalPhaseGate)`


In [77]:
print(optimized_circuit)

(0, 0): ───PhXZ(a=0.5,x=0.5,z=0)────@───PhXZ(a=0.5,x=0.5,z=-1)────M('m')───
                                    │                             │
(0, 1): ───PhXZ(a=0.5,x=-0.5,z=0)───@───PhXZ(a=-0.5,x=0.5,z=-1)───M────────


In [93]:
circuit.all_qubits().

frozenset({cirq.GridQubit(0, 0), cirq.GridQubit(0, 1)})

In [94]:
backend = cirq.CliffordSimulator()

In [100]:
cirq.CliffordSimulator().noise

cirq.NO_NOISE